In [ ]:
import os, sys, shutil, scipy
import numpy as np
import pandas as pd
from scipy.spatial import distance
from sklearn.decomposition import PCA

path = os.getcwd()

if 'Young_Research' in path:
    ffpath   = "C:\\Users\\Young_Research\\Documents\\Rutgers\\Research\\2020_PHY_MSc\\data\\forcefields"
    inpath   = "C:\\Users\\Young_Research\\Documents\\Rutgers\\Research\\2020_PHY_MSc\\data\\initial_conditions"
else:
    ffpath   = "/home/rty10/Documents/Experiments/Optimizations/Opt_ForceFields"
    inpath   = ""


In [ ]:
def gyration(filename):
    df     = pd.DataFrame(columns=['x','y','z'])
    infile = open(filename, 'r')
    rfdata = infile.readlines()
    infile.close()
    rfdata = [i.rstrip('\n').split() for i in rfdata]
    N = int(rfdata[0][0])
    for j in range(0, N):
        df.at[j, ['x','y','z']]  = rfdata[ 5*j + 2]
    df = df.astype('float')
    df['rg'] = (df.x-df.x.mean())**2 + (df.y-df.y.mean())**2 + (df.z-df.z.mean())**2
    R = np.sqrt( (df.rg).mean() )
    del rfdata, df
    return R

In [ ]:
Ncirc = 150
ematrix_dict = {0:'tilt', 1:'roll', 2:'twist', 3:'shift', 4:'slide', 5:'rise'}
incons = ['pc14','pc15','or15']
inseqs = ['col'+str(i).zfill(2) for i in range(5, 10, 1)]

circles= []
[circles.append(filename.split('.')[0]) for filename in os.listdir(path+'/opt_log')]


In [ ]:
df = pd.DataFrame()

for circ in sorted(circles):
    infile = open('opt_log/'+circ+'.log', 'r')
    indata = infile.readlines()
    indata = [i.rstrip('\n') for i in indata]
    infile.close()
    
    df.at[circ, 'seq']        = circ.split('_')[0]
    df.at[circ, 'incon']      = circ.split('_')[1]
    df.at[circ, 'forcefield'] = circ.split('_')[2]
    df.at[circ, 'tot_bp']     = Ncirc
    
    #Specific to set02 data- changes to intrinsic twist
    df.at[circ, 'ffstate']    = circ.split('_')[2].split('-')[0]
    df.at[circ, 'bpturn']     = float(circ.split('-')[1])/10
    df.at[circ, 'two']        = 360 / (float(int(circ.split('-')[1])/10))
    
    for i in range(0, len(indata)):
        if 'initial energy:' in indata[i]:
            df.at[circ, 'eo'] = float( indata[i].split(':')[1] )
        elif 'final energy:' in indata[i]:
            df.at[circ, 'eopt'] = float( indata[i].split(':')[1] )
    ematrix = indata[-7:-1]
    for i in range(0, len(ematrix)):
        ematrix[i] = ematrix[i].replace('{', '').replace('}', '').split(',')
        for j in range(0, len(ematrix[i])):
            ematrix[i][j] = float(ematrix[i][j])
        df.at[circ, 'eopt-'+ematrix_dict[i]] = ematrix[i][i]
    del indata[:]

    infile = open('opt_topology/topo_'+circ+'.txt', 'r')
    indata = infile.readlines()
    indata = [i.rstrip('\n') for i in indata]
    infile.close()
    indata = indata[-4:]
    for i in range(0, len(indata)):
        if 'Wr' in indata[i]:
            df.at[circ, 'Wr'] = float( indata[i].split('=')[1] )
        elif 'Tw' in indata[i]:
            df.at[circ, 'Tw'] = float( indata[i].split('=')[1] )
        elif 'Lk ' in indata[i]:
            df.at[circ, 'Lk'] = int( indata[i].split('=')[1] )
    del indata[:]
    
    radius = gyration('opt_refframe/'+circ+'.dat')
    df.at[circ, 'Rg'] = radius

df.tot_bp = df.tot_bp.astype(int)
df = df.sort_values(by=['incon','seq'])

df.to_csv("data_col150_set02")

del df

In [ ]:
df = pd.read_csv("data_col150_set02", index_col=0)
df.head()

In [ ]:
df.loc[(df.incon=='pc14')&(df.ffstate=='1st')]

In [ ]:
del df